In [1]:
import pandas as pd
import numpy as np
import requests
import time
import re, string
import unicodedata
import six
from MessageClearFunctions import *

In [5]:
# leitura e pré-processamento dos dados
data = pd.read_csv('localizawa_novembro.csv',sep='|',encoding='utf-8')
data[['SequentialId','Content']].to_csv('localizawa_novembro_full.csv',sep='|',encoding='utf-8',index=False)
clear_data = data[data['Content'].apply(is_significant) & data['Content'].apply(is_trash)]
clear_data['Content'] = clear_data['Content'].apply(pre_processing)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [3]:
requests_01 = smalltalk_requests_without_normalization(clear_data, 5, 'localizawa')
requests_02 = smalltalk_requests_with_normalization(clear_data, 5, 'localizawa')

Process finished! Time elapsed = 13.365516662597656 seconds
Process finished! Time elapsed = 11.910441398620605 seconds
Process finished! Time elapsed = 14.183250427246094 seconds
Process finished! Time elapsed = 11.41422152519226 seconds
Process finished! Time elapsed = 12.473191022872925 seconds
Process finished! Time elapsed = 12.30746340751648 seconds
Process finished! Time elapsed = 11.733430624008179 seconds
Process finished! Time elapsed = 11.860683917999268 seconds
Process finished! Time elapsed = 11.708279371261597 seconds
Process finished! Time elapsed = 11.51487421989441 seconds


In [4]:
t1 = []
for request in requests_01:
    begin = time.time()
    t1.append(converting_response_from_API(request))
    end = time.time()
    print('Process finished! Time elapsed = ' + str((end - begin)) +' seconds')
    
t2 = []
for request in requests_02:
    begin = time.time()
    t2.append(converting_response_from_API(request))
    end = time.time()
    print('Process finished! Time elapsed = ' + str((end - begin)) +' seconds')

Process finished! Time elapsed = 8.131179571151733 seconds
Process finished! Time elapsed = 8.868100643157959 seconds
Process finished! Time elapsed = 7.4906165599823 seconds
Process finished! Time elapsed = 7.14587664604187 seconds
Process finished! Time elapsed = 6.853852987289429 seconds
Process finished! Time elapsed = 6.774760007858276 seconds
Process finished! Time elapsed = 6.904479742050171 seconds
Process finished! Time elapsed = 7.539159774780273 seconds
Process finished! Time elapsed = 7.050660610198975 seconds
Process finished! Time elapsed = 7.115802764892578 seconds


In [7]:
api_result1 = pd.DataFrame(data = None)
for dataframe in t1:
    api_result1 = api_result1.append(dataframe).reset_index(drop = True)
    
api_result2 = pd.DataFrame(data = None)
for dataframe in t2:
    api_result2 = api_result2.append(dataframe).reset_index(drop = True)

In [8]:
begin = time.time()
api_result1['RelevantInput'] = api_result1['RelevantInput'].apply(remove_whatsapp_emojis)
api_result1['CleanedInput'] = api_result1['CleanedInput'].apply(remove_whatsapp_emojis)
api_result1['MarkedInput'] = api_result1['MarkedInput'].apply(remove_whatsapp_emojis)

api_result1['RelevantInput'] = api_result1['RelevantInput'].apply(str_strip)
api_result1['CleanedInput'] = api_result1['CleanedInput'].apply(str_strip)
end = time.time()

print('Process finished! Time elapsed = ' + str((end - begin)) +' seconds')

Process finished! Time elapsed = 139.48338985443115 seconds


In [9]:
begin = time.time()
api_result2['RelevantInput'] = api_result2['RelevantInput'].apply(remove_whatsapp_emojis)
api_result2['CleanedInput'] = api_result2['CleanedInput'].apply(remove_whatsapp_emojis)
api_result2['MarkedInput'] = api_result2['MarkedInput'].apply(remove_whatsapp_emojis)

api_result2['RelevantInput'] = api_result2['RelevantInput'].apply(str_strip)
api_result2['CleanedInput'] = api_result2['CleanedInput'].apply(str_strip)
end = time.time()

print('Process finished! Time elapsed = ' + str((end - begin)) +' seconds')

Process finished! Time elapsed = 141.4298071861267 seconds


In [15]:
final_data = api_result2
final_data['MarkedInput'] = api_result1['MarkedInput']

In [16]:
final_data.to_csv('localizawa_test.csv',encoding='utf-8',sep='|',index=False)

In [17]:
final_data = final_data[final_data['RelevantInput'].apply(is_significant) & 
                        ~final_data['RelevantInput'].apply(message_is_empty) &
                        ~final_data['RelevantInput'].apply(is_bot_test) &
                        ~final_data['RelevantInput'].apply(is_uri) &
                        ~final_data['RelevantInput'].apply(is_numeric)]
final_data.to_csv('localizawa_test.csv',encoding='utf-8',sep='|',index=False)   

In [19]:
final_data[final_data['RelevantInput'].apply(is_numeric)]['RelevantInput'].values

array([], dtype=object)

In [21]:
def has_word_in_message(msg):
    if type(msg) is str:
        return ' cpf ' in msg
    else:
        return False

In [28]:
np.array_split(data[['SequentialId','Content']].head(), 5)

[   SequentialId Content
 0     290410271    oiii,    SequentialId             Content
 1     280691796  requisitos aluguel,    SequentialId             Content
 2     280697119  requisitos aluguel,    SequentialId Content
 3     280691490      oi,    SequentialId                             Content
 4     231821173  o que preciso para alugar um carro]